In [1]:
import sys
import os
import torch
from torch import optim
from time import time

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.8 from "/Users/calummcmeekin/miniconda3/envs/git_tile2vec/bin/python"
  * The NumPy version is: "1.19.5"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: dlopen(/Users/calummcmeekin/miniconda3/envs/git_tile2vec/lib/python3.8/site-packages/numpy/core/_multiarray_umath.cpython-38-darwin.so, 2): Symbol not found: __gfortran_concat_string
  Referenced from: /Users/calummcmeekin/miniconda3/envs/git_tile2vec/lib/libcblas.3.dylib
  Expected in: /Users/calummcmeekin/m

In [2]:
tile2vec_dir = '/atlas/u/swang/software/GitHub/tile2vec'
sys.path.append('../')
sys.path.append(tile2vec_dir)

In [3]:
from src.datasets import TileTripletsDataset, GetBands, RandomFlipAndRotate, ClipAndScale, ToFloatTensor, triplet_dataloader
from src.tilenet import make_tilenet

In [4]:
from src.training import prep_triplets, train_triplet_epoch

# Step 1. Download triplets from bucket

Using the download link, unzip triplets into the directory /tile2vec/data/triplets.

# Step 2. Set up dataloader

In [5]:
# Environment stuff
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
cuda = torch.cuda.is_available()


Set up the dataloader for training.

In [7]:
# Change these arguments to match your directory and desired parameters
img_type = 'naip'
tile_dir = '/Users/calummcmeekin/Documents/GitHub/MInf-Project/SatelliteSegmentation/tile2vec/data/toa_triplets'
#tile_dir = '/Users/calummcmeekin/Downloads/tile2vec-master-original/data/triplets/' 
bands = 3
augment = True
batch_size = 50
shuffle = True
num_workers = 4
n_triplets = 58202

In [8]:
dataloader = triplet_dataloader(img_type, tile_dir, bands=bands, augment=augment,
                                batch_size=batch_size, shuffle=shuffle, num_workers=num_workers, 
                                n_triplets=n_triplets, pairs_only=True)
print('Dataloader set up complete.')

Dataloader set up complete.


# Step 3. Set up TileNet

In [9]:
in_channels = bands
z_dim = 512

In [10]:
TileNet = make_tilenet(in_channels=in_channels, z_dim=z_dim)
TileNet.train()
if cuda: TileNet.cuda()
print('TileNet set up complete.')

TileNet set up complete.


Set up optimizer.

In [11]:
lr = 1e-3
optimizer = optim.Adam(TileNet.parameters(), lr=lr, betas=(0.5, 0.999))

# Step 4. Train model!

In [12]:
epochs = 100
margin = 10
l2 = 0.01
print_every = 1000
save_models = True


Define the directory for saving models.

In [13]:
model_dir = '/Users/calummcmeekin/Documents/GitHub/MInf-Project/SatelliteSegmentation/tile2vec/models'
if not os.path.exists(model_dir): os.makedirs(model_dir)

In [14]:
t0 = time()
with open("results_fn", 'w') as file:

    print('Begin training.................')
    for epoch in range(0, epochs):
        (avg_loss, avg_l_n, avg_l_d, avg_l_nd) = train_triplet_epoch(
            TileNet, cuda, dataloader, optimizer, epoch+1, margin=margin, l2=l2,
            print_every=print_every, t0=t0)

Begin training.................
Epoch 1: [1000/58202 (2%)], Avg loss: 12.0928
Epoch 1: [2000/58202 (3%)], Avg loss: 9.9838
Epoch 1: [3000/58202 (5%)], Avg loss: 9.0502
Epoch 1: [4000/58202 (7%)], Avg loss: 8.9914
Epoch 1: [5000/58202 (9%)], Avg loss: 8.9708
Epoch 1: [6000/58202 (10%)], Avg loss: 8.7959
Epoch 1: [7000/58202 (12%)], Avg loss: 8.8445
Epoch 1: [8000/58202 (14%)], Avg loss: 8.5304
Epoch 1: [9000/58202 (15%)], Avg loss: 8.6283
Epoch 1: [10000/58202 (17%)], Avg loss: 8.5290
Epoch 1: [11000/58202 (19%)], Avg loss: 8.3055
Epoch 1: [12000/58202 (21%)], Avg loss: 8.4373
Epoch 1: [13000/58202 (22%)], Avg loss: 8.1588
Epoch 1: [14000/58202 (24%)], Avg loss: 8.5661
Epoch 1: [15000/58202 (26%)], Avg loss: 8.0545
Epoch 1: [16000/58202 (27%)], Avg loss: 8.2266


KeyboardInterrupt: 

In [ ]:
# Save model after last epoch
if save_models:
    model_fn = os.path.join(model_dir, 'TileNet_epoch100_toa_data.ckpt')
    torch.save(TileNet.state_dict(), model_fn)